> introduction

- QIIME 2를 사용하여 칠레 북부의 아타카마 사막에서 토양 샘플을 분석해보자.
  - 아타카마 사막은 지구상에서 가장 건조한 지역 중 하나
  - 이 극심한 건조에도 불구하고 토양에 미생물이 살고 있음
  - 이 연구에서 수집한 토양 미생물 군집은 평균 토양 상대 습도가 고도와 양의 상관관계가 있는 두 개의 동서 단면인 Baquedano와 Yungay를 따른다. 
     - 높은 고도는 덜 건조하므로 평균 토양 상대 습도가 더 높다. 
     - 이 횡단면을 따라 각 사이트에서 구덩이를 파고 각 구덩이의 세 깊이에서 토양 샘플을 수집함. 

> ### Explore the step

```
- tutorial을 진행할 dir을 만들자.
- 그리고 그 dir로 이동하자

code;

mkdir qiime2-atacama-tutorial
cd qiime2-atacama-tutorial

- This "sample-metadata.tsv" file is used throughout the rest of the tutorial

code;

wget \
  -O "sample-metadata.tsv" \
  "https://data.qiime2.org/2022.11/tutorials/atacama-soils/sample_metadata.tsv"

- Download the multiplexed reads. 
- Download three fastq.gz files, 
  - corresponding to the forward, reverse, and barcode (i.e., index) reads. 

- multiplexed reads data들을 download

code;

mkdir emp-paired-end-sequences

# forward
wget \
  -O "emp-paired-end-sequences/forward.fastq.gz" \
  "https://data.qiime2.org/2022.11/tutorials/atacama-soils/10p/forward.fastq.gz"
  
# reverse
wget \
  -O "emp-paired-end-sequences/reverse.fastq.gz" \
  "https://data.qiime2.org/2022.11/tutorials/atacama-soils/10p/reverse.fastq.gz"
  
# barcodes
wget \
  -O "emp-paired-end-sequences/barcodes.fastq.gz" \
  "https://data.qiime2.org/2022.11/tutorials/atacama-soils/10p/barcodes.fastq.gz"



```

- sequence data들을 다운로드 받음
  - 이를 분석하기 위해서 artifact로 변환 필요..
```
qiime tools import \
   --type EMPPairedEndSequences \
   --input-path emp-paired-end-sequences \
   --output-path emp-paired-end-sequences.qza
```
- 이때, --type EMPPairedEndSequences이 의미하는 건? artifact의 타입!
- input? 세 개의 파일이 들어있는 폴더 자체를 넣어주는 거 같음...
- 그리고 output으로 emp-paired-end-sequences.qza이 나옴
  - 이 output 파일의 artifact 타입이 EMPPairedEndSequences

- 이제 demultiplex the sequence reads
  - 이 과정을 위해 필요한 거?
  - metadata file!
  - 이 파일에서 샘플별 바코드가 포함된 열을 지정해야 한다. 
  - 이 경우 해당 열의 이름은 바코드 시퀀스이다. 
```
qiime demux emp-paired \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --p-rev-comp-mapping-barcodes \
  --i-seqs emp-paired-end-sequences.qza \
  --o-per-sample-sequences demux-full.qza \
  --o-error-correction-details demux-details.qza
  ```

- subsample??
  - Let’s subsample the data. 
  - We will perform this subsampling in this tutorial for two reasons
    - one, to speed up the tutorial run time
    - two, to demonstrate the functionality
```
qiime demux subsample-paired \
  --i-sequences demux-full.qza \
  --p-fraction 0.3 \
  --o-subsampled-sequences demux-subsample.qza

qiime demux summarize \
  --i-data demux-subsample.qza \
  --o-visualization demux-subsample.qzv
```

---

- Let’s take a look at the summary in demux-subsample.qzv.
- In the “Per-sample sequence counts” table on the “Overview” tab, there are 75 samples in the data. 
  - If we look at the last 20 or so rows in the table, though, we will observe that many samples have fewer than 100 reads in them - let’s filter those samples out of the data:
  - 어떻게?
  ```
  qiime tools export \
  --input-path demux-subsample.qzv \
  --output-path ./demux-subsample/

qiime demux filter-samples \
  --i-demux demux-subsample.qza \
  --m-metadata-file ./demux-subsample/per-sample-fastq-counts.tsv \
  --p-where 'CAST([forward sequence count] AS INT) > 100' \
  --o-filtered-demux demux.qza
  ```

- Next, we’ll look at the sequence quality based on ten-thousand randomly selected reads from the subsampled and filtered data, and then denoise the data. 
- The plot on the left presents the quality scores for the forward reads, and the plot on the right presents the quality scores for the reverse reads. 
- We’ll use these plots to determine what trimming parameters we want to use for denoising with DADA2, and then denoise the reads using dada2 denoise-paired.

- In this example we have 150-base forward and reverse reads. Since we need the reads to be long enough to overlap when joining paired ends, the first thirteen bases of the forward and reverse reads are being trimmed, but no trimming is being applied to the ends of the sequences to avoid reducing the read length by too much. 
- In this example, the same values are being provided for --p-trim-left-f and --p-trim-left-r and for --p-trunc-len-f and --p-trunc-len-r, but that is not a requirement.
```
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left-f 13 \
  --p-trim-left-r 13 \
  --p-trunc-len-f 150 \
  --p-trunc-len-r 150 \
  --o-table table.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-denoising-stats denoising-stats.qza
  ```
  

- At this stage, you will have artifacts containing the feature table corresponding feature sequences, and DADA2 denoising stats. You can generate summaries of these as follows.
```
qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv

qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv
  ```

----

> DADA2로 quality filtering 완료, 해당 데이터를 visualization 해보자

\- feature-table summarize
  - Information on how many sequences are associated with each sample and with each feature, histogram of those distributions, and some related summary statistics.
  - 각 샘플에서 얼마나 많은 시퀀스들이 각 feature들에 대해서 어느정도 유사한지, 그 분포를 히스토그램으로, 그리고 통계량으로 관측.
- feature-table tabulate-seqs
  - 각 시퀀스의 feature ID의 mapping 제공
  - provide links to BLAST each sequence against the NCBI nt database
    - ???
    - dada2로 필터링 -> feature-table tabulate-seqs -> BLAST라는 알고리즘을 이용하여 NCBI라는 데이터베이스를 참고하여 align, 비교대조한다. 

```
  qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv
qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv
```

----

> Generate a tree for phylogenetic diversity analyses - 계통발생적 다양성 분석을 위한 tree 생성

    QIIME supports several phylogenetic diversity metrics, including <Faith’s Phylogenetic Diversity> and <weighted and unweighted UniFrac>
    
    필요한 거?
    feature table + a rooted phylogenetic tree
    
    해당 정보는  Phylogeny[Rooted] QIIME2 artifact에 형성된다. 
    
    To generate a phylogenetic tree, we will use align-to-tree-mafft-fasttree pipeline from the q2-phylogeny plugin.

- first step
  - the pipeline uses the mafft program to perform a multiple sequence alignment of the sequences in our FeatureData[Sequence] to create a FeatureData[AlignedSequence] QIIME 2 artifact.
- next step
  - Next, the pipeline masks (or filters) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree. 매우 가변적인 위치를 제거한다. 이러한 가변적인 위치는 계통발생학적 tree에 노이즈로 취급되기 때문에 제거하는 것.
    - Following that, the pipeline applies FastTree to generate a phylogenetic tree from the masked alignment.
    - masked alignment로부터 phylogenetic tree를 만들기 위해서 FastTree 프로그램을 이용한다. 
    - FastTree 프로그램은 unrooted tree를 생성하고  so in the final step in this section midpoint rooting is applied to place the root of the tree at the midpoint of the longest tip-to-tip distance in the unrooted tree.
    - 내 생각?
    - rooted tree의 midpoint rooting이 unrooted tree의 midpoint rooting으로 사용되는 듯???
    

> Alpha and beta diversity analysis

- QIIME 2’s diversity analyses are available through the q2-diversity plugin
  - 이 스크립트에서 제공해야 하는 중요한 매개변수는 --p-sampling-depth이며, 이는 짝수 샘플링(즉, 희박화)의 깊이이다. 
  - diversity metric은 다양한 샘플의 다양한 깊이에 민감하기 때문에 이 스크립트는 각 샘플의 카운트를 이 매개변수에 제공된 값으로 무작위 서브 샘플링한다. 
  - 예를 들어, --p-sampling-depth 500을 제공하는 경우 이 단계는 결과 테이블의 각 샘플이 총 500개가 되도록 교체없이 각 샘플의 카운트를 서브샘플링한다. 
    - 이 값보다 작으면 해당 샘플은 다양성 분석에서 제외된다 .
    - 이 값은 table.qzv 파일에 제시된 정보를 검토하여 선택하는 것이 좋다.
    - 가능한 한 적은 수의 샘플을 제외하면서 가능한 높은 값을 선택해야 한다. 
      - 샘플 당 더 많은 시퀀스 유지
      -  Losing a disproportionate number of samples from one metadata category is not ideal.

- 그런데 ...feature table을 희박화 한다는 게 뭘 의미?
  - 